# Money and death



We return to the death penalty.

In [ ]:
# Array library.
import numpy as np

# A Numpy random number generator.
rng = np.random.default_rng()

# Data frames library.
import pandas as pd
# Safe setting for Pandas.  Needs Pandas version >= 1.5.
pd.set_option('mode.copy_on_write', True)

# Set up plotting
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
# Load the OKpy test library and tests.
from client.api.notebook import Notebook
ok = Notebook('money_death.ok')

In this case, we are going to analyze whether people with higher incomes are
more likely to favor the death penalty.

To do this, we are going to analyze the results from a sample of the US
[General Social Survey](http://www.gss.norc.org) from 2002.

Make sure you have the data file [GSS2002.csv](https://lisds.github.io/textbook/data/GSS2002.csv) in the same directory as this notebook.  This should be true if you unpacked the exercise zip file.

In [ ]:
# Read the data into a data frame
gss = pd.read_csv('GSS2002.csv')
# Show the first 5 rows.
gss.head()

Each row corresponds to a single respondent.

Show the column names:

In [ ]:
gss.columns

We want to work with only two columns from this data frame.  These are
"Income", and "DeathPenalty".

"Income" gives the income bracket of the respondent.  "DeathPenalty" is the
answer to a question about whether they "Favor" or "Oppose" the death penalty.

Start by making a data frame `money_death` that only has the columns "Income" and "DeathPenalty" from `gss`.

**Hint**: make a new empty data frame called `money_death`.  Get the columns
"Income" and "DeathPenalty" from the `gss` data frame, and insert them into
`money_death`.  See the [data frames, series, arrays page](https://lisds.github.io/textbook/data-frames/df_series_arrays).

In [ ]:
money_death = ...
# Show the result
money_death.head()

In [ ]:
_ = ok.grade('q_1_money_death')

There are many missing question responses, indicated by `NaN`.  To make our
life easier, we drop the respondents who didn't specify an income bracket, and
those who did not give an answer to the death penalty question.  Pandas
`dropna` method of the data frame does this job. It drops all rows that have
any missing values in the row.  Check the help for the `money_death.dropna`
method with `money_death.dropna?` in a new cell.

In [ ]:
complete_md = ...
complete_md.head()

In [ ]:
_ = ok.grade('q_2_complete_md')

Put the income column of `complete_md` into its own variable:

In [ ]:
income = ...
income.head()

In [ ]:
_ = ok.grade('q_3_income')

Use the `value_counts` method of the `income` Series to show the unique values and their counts.

In [ ]:
income.value_counts()

The elements in this series are strings.  We want to get income as a number.
We estimate this by *recoding* the "Income" column.  We replace the string,
giving the income bracket, with the average of the minimum and maximum in the
range.

We can do this with a *recoder function*.  We have not covered writing your own
functions yet, so do not worry about the details of this function.

In [ ]:
# Just run this cell
def recode_income(value):
    if value == 'under 1000':
        return 500
    low_str, high_str = value.split('-')
    low, high = int(low_str), int(high_str)
    return np.mean([low, high])

Here is what the recoder function gives with the lowest income bracket.

In [ ]:
recode_income('under 1000')

Here is what it gives for a higher bracket:

In [ ]:
recode_income('90000-109999')

The cell below uses this function to recode the "Income" strings into numbers.
Again, we have not covered the `apply` method yet, so don't worry about the
details.

In [ ]:
# Run this cell.
income_ish = income.apply(recode_income)
income_ish.head()

In [ ]:
# This test makes sure you've run the cells above correctly.
_ = ok.grade('q_4_income_ish')

Next put the column containing the answers to the death penalty question into
its own variable `death`:

In [ ]:
death = ...
# Show the unique values with their counts.
death.value_counts()

In [ ]:
_ = ok.grade('q_5_death')

In the next few cells, you make two new Pandas Series.  One, `favor_income`,
contains the recoded income values `income_ish` corresponding to respondents
who have a `death` value of `Favor`.

The other, `oppose_income`, has the recoded income values corresponding to those with `Oppose` values in `death`.

**Hint** - as ever, make a Boolean series of True / False values corresponding
to rows you want.  Use that series to select the rows from `income_ish`.

In [ ]:
favor_income = ...
# Show a histogram of the in-favor income values.
favor_income.hist();

In [ ]:
_ = ok.grade('q_6_favor_income')

Likewise select incomes for those opposed.   Show the distribution.

In [ ]:
oppose_income = ...
# Show a histogram of the oppose income values.
oppose_income.hist();

In [ ]:
_ = ok.grade('q_7_oppose_income')

Calculate the difference in mean income between the groups.  This is the
difference we observe.

In [ ]:
actual_diff = ...
actual_diff

In [ ]:
_ = ok.grade('q_8_actual_diff')

We want to know whether this difference in income is compatible with random
sampling. That is, we want to know whether a difference this large is
plausible, if the incomes are in fact random samples from the same population.

To estimate how variable the mean differences can be, for such random sampling,
we simulate this sampling by pooling the income values that we have, from the
two groups, and the permuting them.

First, we get the number of respondents in favor of the death penalty.

In [ ]:
n_favor = ...
n_favor

In [ ]:
_ = ok.grade('q_9_n_favor')

Then we pool the income values for the in-favor and oppose groups.

In [ ]:
# Concatenate the in-favor and opposed incomes.
pooled = np.concatenate([favor_income, oppose_income])
# Show the first 10 values before shuffling.
pooled[:10]

To do the random sampling we permute the values, so the `pooled` vector is a
random mixture of the two groups.

In [ ]:
shuffled = rng.permutation(pooled)
# Show the first 10 values after shuffling.
shuffled[:10]

Treat the first `n_favor` observations from this shuffled vector as our
simulated in-favor group.  The rest are our simulated oppose
group.

In [ ]:
fake_favor = shuffled[:n_favor]
fake_oppose = shuffled[n_favor:]

Calculate the difference in means for this simulation.

In [ ]:
fake_diff = np.mean(fake_favor) - np.mean(fake_oppose)
fake_diff

Now it is your turn.   Do this simulation 10000 times, to build up the
distribution of differences compatible with random sampling.

In [ ]:
fake_differences = ...
for i in np.arange(10000):
    #- Permute the pooled incomes
    shuffled = rng.permutation(pooled)
    #- Make a fake favor sample
   fake_favor = ...
    #- Make a fake opposed sample
   fake_oppose = ...
    #- Calculate the mean difference for the fake samples
   fake_diff = ...
    #- Put the mean difference into the fake_differences array.
   fake_differences...
# Show the first 10 fake differences
fake_differences[:10]

In [ ]:
_ = ok.grade('q_10_fake_differences')

When you have that working, do a histogram of the differences.

In [ ]:
plt.hist(fake_differences)

You can get an idea of where the actual difference we saw sits on this
histogram, and therefore how likely that difference is, assuming the incomes
come from the same underlying population of incomes.

To be more specific, show the proportion of the differences you calculated that
were greater than or equal to the actual difference.

In [ ]:
p_fake_ge_actual = ...
p_fake_ge_actual

In [ ]:
_ = ok.grade('q_11_p_fake_ge_actual')

This proportion gives an estimate of the probability of seeing a difference
this large, if the incomes all come from the same underlying population.

## Done

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]